In [1]:
import sys
import os

# 指定Python解释器的搜索路径
notebook_path = os.path.abspath('')
project_root = os.path.dirname(notebook_path)
sys.path.append(project_root)

import gxipy as gx
from PIL import Image
import datetime
import time

## 工具类

In [3]:
def print_promotion(message):
    print("")
    print("-------------------------------------------------------------")
    if message is not None:
        print(message)
    print("-------------------------------------------------------------")
    print("")
    

## 采集程序

### 创建设备管理对象

In [4]:
device_manager = gx.DeviceManager()

### 枚举设备

In [5]:
dev_num, dev_info_list = device_manager.update_device_list()
if dev_num == 0:
    print("Number of enumerated devices is 0")
else:
    print_promotion("创建设备成功，设备数量为:%d" % dev_num)



-------------------------------------------------------------
创建设备成功，设备数量为:2
-------------------------------------------------------------



### 相机参数设置

In [6]:
def camera_params_setting(cam, framerate_set):
    """
    cam: 相机对象
    framerate_set: 相机拍摄帧率
    """
    width_set = 2448  # 设置分辨率宽
    height_set = 2048  # 设置分辨率高

    # 设置宽和高
    cam.Width.set(width_set)
    cam.Height.set(height_set)

    # 设置连续采集
    # cam.TriggerMode.set(gx.GxSwitchEntry.ON)  # 设置外触发模式
    # cam.TriggerSource.set(gx.GxTriggerSourceEntry.LINE1)  # 设置触发源
    # cam.ExposureTime.set(10000.0)  # 设置曝光时间
    # cam.Gain.set(10.0)  # 设置增益

    cam.AcquisitionFrameRateMode.set(gx.GxSwitchEntry.ON)
    cam.AcquisitionFrameRate.set(framerate_set)
    # 自动白平衡
    cam.BalanceWhiteAuto.set(gx.GxAutoEntry.CONTINUOUS)
    # 自动曝光
    cam.ExposureAuto.set(gx.GxAutoEntry.CONTINUOUS)  # 或者gx.GxAutoEntry.ONCE


### 图像采集设置

In [7]:
def acquisition_setting(cam, num, framerate_set):
    """
    cam: 相机对象
    num: 采集的图像数量
    framerate_set: 设置的采集帧率
    """
    cam.stream_on()

    # 采集图像
    for i in range(num):
        framerate_get = cam.CurrentAcquisitionFrameRate.get()  # 获取当前采集的帧率
        raw_image = cam.data_stream[0].get_image()  # 打开第0通道数据流
        if raw_image is None:
            print("获取彩色原始图像失败.")
            continue
        rgb_image = raw_image.convert("RGB")  # 从彩色原始图像获取RGB图像
        if rgb_image is None:
            continue
        # rgb_image.image_improvement(color_correction_param, contrast_lut, gamma_lut)  # 实现图像增强
        numpy_image = rgb_image.get_numpy_array()  # 从RGB图像数据创建numpy数组
        if numpy_image is None:
            continue
        img = Image.fromarray(numpy_image, 'RGB')  # 展示获取的图像
        # img.show()
        mtime = datetime.datetime.now().strftime('%Y-%m-%d_%H_%M_%S')
        img.save(r"D:\FileDevelop\DevelopTools\Pycharm\DaHengImage\Galaxy\CaptureByScripts\\" + mtime + str("-") + str(i) + ".jpg")  # 保存图片到本地
        print("Frame ID: %d   Height: %d   Width: %d   framerate_set:%dfps   framerate_get:%dfps"
                % (raw_image.get_frame_id(), raw_image.get_height(), raw_image.get_width(), framerate_set,
                    framerate_get))  # 打印采集的图像的高度、宽度、帧ID、用户设置的帧率、当前采集到的帧率

### 图像采集过程

In [8]:
framerate_set = 20  # 设置帧率
num = 10  # 采集帧率次数（为调试用，可把后边的图像采集设置成while循环，进行无限制循环采集）

for dev_info in dev_info_list:
    sn = dev_info.get("sn")
    cam = device_manager.open_device_by_sn(sn)
    # 如果是黑白相机，则不支持
    if cam.PixelColorFilter.is_implemented() is False:  # is_implemented判断枚举型属性参数是否已实现
        print("该示例不支持黑白相机.")
        cam.close_device()
    else:
        print_promotion("打开彩色摄像机成功，SN号为：%s，开始采集......" % sn)
    
    # 设置相机参数
    camera_params_setting(cam, framerate_set)

    # 等待五秒，调整白平衡和曝光
    time.sleep(5)

    # 相机采集设置
    acquisition_setting(cam, num, framerate_set)
    
    # 停止采集并关闭相机
    cam.stream_off()
    cam.close_device()
    print_promotion("摄像机 {} 已经停止采集，设备已经关闭！".format(sn))




-------------------------------------------------------------
打开彩色摄像机成功，SN号为：LKS24040027，开始采集......
-------------------------------------------------------------

Frame ID: 1   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 2   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 3   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 4   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 5   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 6   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 7   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 8   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 9   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get:3fps
Frame ID: 10   Height: 2048   Width: 2448   framerate_set:20fps   framerate_get: